<a href="https://colab.research.google.com/github/greyhound101/IEEE-CIS-Fraud/blob/master/Cleaning_Preprocessingand_adding_columns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Getting data

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "tapaskd123" # username from the json file
os.environ['KAGGLE_KEY'] = "aba8dc1f085221111d925003fe5a88ed" # key from the json file
!kaggle competitions download -c ieee-fraud-detection

test_identity.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_transaction.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_transaction.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_identity.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


Loading drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


Reading datasets

In [ ]:
import pandas as pd
import numpy as np
trn=pd.read_csv('train_transaction.csv.zip')
tst=pd.read_csv('test_transaction.csv.zip')
trn_idn=pd.read_csv('train_identity.csv.zip')
tst_idn=pd.read_csv('test_identity.csv.zip')

Merging datasets

In [ ]:
train=trn.merge(trn_idn,on='TransactionID',how='left')
test=tst.merge(tst_idn,on='TransactionID',how='left')
import gc
del([trn,tst,trn_idn,tst_idn])
gc.collect()

11

**Handling nan**

In [ ]:
all_columns=[]

In [ ]:
a=list(train.filter(regex='id'))
b=list(test.filter(regex='id'))
dk={}
for i,j in zip(a,b):
  dk[j]=i
test.rename(columns=dk,inplace=True)

Checking nan

In [ ]:
ls=list(test)
ln=len(train)
dk={}
for col in ls:
  percent=100*train[col].isna().sum()/ln
  dk[col]=percent
df=pd.DataFrame(dk,index=[1])
df=df.T
df.columns=['nan']
df['columns']=df.index
df=df.reset_index(drop=True)
df

,nan,columns
0,0.000000,TransactionID
1,0.000000,TransactionDT
2,0.000000,TransactionAmt
3,0.000000,ProductCD
4,0.000000,card1
...,...,...
428,76.126088,id_36
429,76.126088,id_37
430,76.126088,id_38
431,76.155722,DeviceType


Dropping columns with greater than 75 percent nan

In [ ]:
dps=df.loc[df['nan']>75]['columns'].values
train=train.drop(dps,1)
test=test.drop(dps,1)
del([df])
gc.collect()

0




*   Get sum of nan for categorical and continuous columns
*   Fill Cs and Ds by -999 other numerical by mean and categorical by 'nan'
*   Get indicator for each column nan 



Getting an indicator column for each column for nan

In [ ]:
ls=list(test)
cat=list(test.select_dtypes(include=object))
for col in ls:
  if train[col].isna().sum()>0:
    all_columns+=[col+'_isna']
    train[col+'_isna']=train[col].isna().astype('uint8')
    test[col+'_isna']=test[col].isna().astype('uint8')

Seperating numerical and categorical type columns

In [ ]:
categorical = ["ProductCD", "card1", "card2", "card3", "card4", "card5", "card6", "addr1", "addr2",
               "P_emaildomain",'P_emaildomain_first','P_emaildomain_second', "R_emaildomain",
              "DeviceInfo", "DeviceType"] + ["id_0" + str(i) for i in range(1, 10)] +\
                ["id_" + str(i) for i in range(10, 39)] + \
                 ["M" + str(i) for i in range(1, 10)]
categorical=set(categorical).intersection(set(list(test)))
numerical = ["TransactionAmt", "dist1", "dist2","day","week","year"] + ["C" + str(i) for i in range(1, 15)]+["D" + str(i) for i in range(1, 16)]+["V" + str(i) for i in range(1,340)]
numerical=list(set(numerical).intersection(set(test)))
all_columns+=numerical

In [ ]:
train[numerical].filter(regex='D').fillna(-999,inplace=True)
test[numerical].filter(regex='D').fillna(-999,inplace=True)

In [ ]:

train[numerical].filter(regex='C').fillna(-999,inplace=True)
test[numerical].filter(regex='C').fillna(-999,inplace=True)

In [ ]:
for col in numerical:
  train[col]=train[col].fillna(train[col].mean())
  test[col]=test[col].fillna(test[col].mean())
for col in categorical:
  train[col]=train[col].fillna('nan')
  test[col]=test[col].fillna('nan')



















**Creating New columns**

Nan count for each types

In [ ]:
def num_cat(df,typ,nme):
  df[nme]=0
  for col in typ:
    try:
      df[nme]+=df[col+'_isna']
    except:
      continue
  return df
all_columns+=['numerical','categorical']
train=num_cat(train,numerical,'numerical')
train=num_cat(train,categorical,'categorical')


test=num_cat(test,numerical,'numerical')
test=num_cat(test,categorical,'categorical')

Seems we can split p_emaildomain into parts before and after '.'

In [ ]:
def second(data):
  return data[data.find('.'):]
def first(data):
  return data[:data.find('.')]
all_columns+=['P_emaildomain_first','P_emaildomain_second']
train['P_emaildomain_first']=train['P_emaildomain'].apply(first)
train['P_emaildomain_second']=train['P_emaildomain'].apply(second)
test['P_emaildomain_first']=test['P_emaildomain'].apply(first)
test['P_emaildomain_second']=test['P_emaildomain'].apply(second)

Getting dummies of top 10 most common categories and label encoding of card1 and id_02 since they have many categories

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
def top_n(categorical,fnl,n):
  cols_name=[]
  for col in tqdm(categorical):
        print(col)
        print('before '+str(fnl[col].nunique()))
        fnl.loc[~fnl[col].isin(list(fnl[col].value_counts().index[:n])),col]='other'
        print('before '+str(fnl[col].nunique()))
        if col!='card1' or col !='id_02':
            fnl_dum=pd.get_dummies(fnl[col]).astype('uint8')
            fnl_dum.columns=list(np.char.add(col,np.asarray(list(fnl_dum)).astype(str)))
            fnl_dum.columns=fnl_dum.columns+'_dum'
            cols_name.extend(list(fnl_dum.columns))
            #fnl=fnl.drop([col],1)
            fnl=pd.concat([fnl,fnl_dum],1)
            del([fnl_dum])
            gc.collect()
        else:
          le=LabelEncoder()
          fnl[col]=le.fit_transform(fnl[col]).astype('uint8')
  return fnl,cols_name
print(train.shape)
print(test.shape)
df,new_cols=top_n(categorical,pd.concat([train,test],0).reset_index(drop=True),50)
all_columns+=new_cols
print(df.shape)

(590540, 436)
(506691, 435)


  0%|          | 0/19 [00:00<?, ?it/s]

M4
before 4
before 4


  5%|▌         | 1/19 [00:00<00:15,  1.20it/s]

P_emaildomain
before 61
before 11


...

M1
before 3
before 3


 95%|█████████▍| 18/19 [00:26<00:01,  1.54s/it]

card1
before 17091
before 11


100%|██████████| 19/19 [00:28<00:00,  1.50s/it]

(1097231, 556)


In [ ]:
train=df.loc[:590539].reset_index(drop=True)
test=df.loc[590540:].reset_index(drop=True)
print(train.shape)
print(test.shape)
del([df])
gc.collect()

(590540, 556)
(506691, 556)


0

Convert TransactionDT to day,month,year

In [ ]:
def time(df):
  df['day']=df['TransactionDT']//86400
  df['week']=df['TransactionDT']//(86400*7)
  df['month']=df['TransactionDT']//(86400*30)
  return df
train=time(train)
test=time(test)
all_columns+=['day','week','month']

Creating new columns

In [ ]:
for i in [1,2,3,4,5,10,11,15]:
  all_columns+=['d_'+str(i)]
  train['d_'+str(i)]=train['day']-train['D'+str(i)]
  test['d_'+str(i)]=test['day']-test['D'+str(i)]

Creating id

In [ ]:
train['id']=''
test['id']=''
all_columns+=['id']
for col in ['C1','C13','P_emaildomain','addr1','card1','d_1']:
  train['id']+=train[col].astype(str)
  test['id']+=test[col].astype(str)

**Final Preprocessing**

Reducing memory used

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
train=reduce_mem_usage(train)
test=reduce_mem_usage(test)
ids=[]

Memory usage of dataframe is 1273.92 MB
Memory usage after optimization is: 662.28 MB
Decreased by 48.0%
Memory usage of dataframe is 1093.04 MB
Memory usage after optimization is: 576.06 MB
Decreased by 47.3%


Standarizing data

In [ ]:
def fill(df,numerical,ids):
  for col in (numerical+ids):
    if df[col].skew()>1:
        df[col]=np.log10(df[col]+1-min(0,min(df[col])))
    mn=df[col].mean()
    std=df[col].std()
    df[col]=(df[col]-mn)/(std+1e-6)
    return df
train=fill(train,numerical,ids)
test=fill(test,numerical,ids)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:

test[all_columns].to_csv(F"/content/gdrive/My Drive/fraud/test.csv")
all_columns+=['isFraud']
train[all_columns].to_csv(F"/content/gdrive/My Drive/fraud/train.csv")

all_columns consists of 
**categorical**
1. id  
2. emaildomain split
3. dummies
4. categorical
5. isna flags
**numerical**

6. date, month  
7. nan sum
8. d
9. numerical

**Takeaway points**
1. Included columns with less than 75 percent nanas
2. Got indicator for nan columns
3. Split emaildomain after the '.'
4. For each categorical columns got dummies of top 10 most frequent value 
5. Since D columns were difference in days created columns by subtracting from days
6. Filled nan of conitnuous columns by mean and for categorical columns by 'nan' and Cs and Ds by -999